In [20]:
import pandas as pd
import sqlite3 as sql
import logging
import pytest

logger = logging.getLogger(__name__)
logging.basicConfig(
        filename=r'C:\Users\vidar\Documents\Skóli\TUC\Data_Science\Projects\Kunskapskontrol_1\log.txt', 
        format='[%(asctime)s][%(levelname)s] %(message)s',
        level=logging.INFO
        )

df1 = pd.read_csv(r'C:\Users\vidar\Documents\Skóli\TUC\Data_Science\Projects\Kunskapskontrol_1\000007N5_20241217-094027.csv', encoding='cp1252')
df2 = pd.read_csv(r'C:\Users\vidar\Documents\Skóli\TUC\Data_Science\Projects\Kunskapskontrol_1\000003PS_20241217-093537.csv', encoding='cp1252')

df2.drop(['typ av IT-tjänst'], axis=1, inplace=True)
df1 = df1.drop(['typ av konsekvens','2023'], axis=1).loc[df1['2021']!='..']

df1['2021'] = df1['2021'].astype(int)

df1 = df1.groupby('redovisningsgrupp').sum()

def fix_regions(df):
        df.rename(index={'Stockholms län': 'Stockholm', 'Norra mellansverige':'Norra Mellansverige', 'Östra mellansverige':'Östra Mellansverige'}, inplace=True)
        
        
fix_regions(df1)
fix_regions(df2)

df3 = df1.merge(df2, 'left', right_on='redovisningsgrupp', left_on='redovisningsgrupp')

df3.rename(columns={'redovisningsgrupp':'Region', '2021_x':'Incidenter 2021', '2021_y':'Utgifter för IT tjänster 2021'}, inplace=True)

con = sql.connect('kunskapskontrol1.db')
df3.to_sql('data om it-utgifter och incidenter', con, if_exists='replace')

8

In [10]:
# Kunskapskontrol 1
import pandas as pd
import sqlite3 as sql
import logging

logger = logging.getLogger(__name__)
logging.basicConfig(
        filename=r'C:\Users\vidar\Documents\Skóli\TUC\Data_Science\Projects\Kunskapskontrol_1\log.txt', 
        format='[%(asctime)s][%(levelname)s] %(message)s',
        level=logging.INFO
        )
# Läsa in csv från SCB, passa på att specifiera encoding som cp1252 annars får man inte svenska bokstaver
try:
    df1 = pd.read_csv(r'C:\Users\vidar\Documents\Skóli\TUC\Data_Science\Projects\Kunskapskontrol_1\000007N5_20241217-094027.csv', encoding='cp1252')
    df2 = pd.read_csv(r'C:\Users\vidar\Documents\Skóli\TUC\Data_Science\Projects\Kunskapskontrol_1\000003PS_20241217-093537.csv', encoding='cp1252')
except FileNotFoundError:
    logger.warning(f'File Not Found, check file path!')
else:
    logger.info(f'CSV file was read')
# rensa df1, droppa 2 kolumner som vi inte behöver samt alla nullvärde i den kolumnen vi håller i.
# SCB använder ".." som nullvärde så pandas .dropna kommer inte funka
# i csv'en är siffrorna klassifierade som textsträngar så vi måste omvändla til int
# Vi är egentligen bara intresserad av total incidenter

def clean_df1(df1):
        df1 = df1.drop(['typ av konsekvens','2023'], axis=1).loc[df1['2021']!='..']
        df1['2021'] = df1['2021'].astype(int)
        df1 = df1.groupby('redovisningsgrupp').sum()
        return df1
# SCB använder ibland inte samma exakta namn för regionerna så vi får döpa om värden i första dataframe så de matchar namnen i den andra
def fix_regions(*dfs):
    for df in dfs:
        df.rename(
            index={'Stockholms län': 'Stockholm',
                   'Norra mellansverige':'Norra Mellansverige',
                   'Östra mellansverige':'Östra Mellansverige'},
            inplace=True
            )
        
# För df2 kan vi droppa den kolumnen eftersom den bara har värdet "Totalt" i alla rader
def clean_df2(df):
    return df2.drop(['typ av IT-tjänst'], axis=1, inplace=True)

df1 = clean_df1(df1)
fix_regions(df1, df2)
df2 = clean_df2(df2)


# Vi kör merge, som tar liknande argument som SQL join, och skapar df3
df3 = df1.merge(df2, 'left', right_on='redovisningsgrupp', left_on='redovisningsgrupp')

df3.rename(
    columns={'redovisningsgrupp':'Region',
             '2021_x':'Incidenter 2021',
             '2021_y':'Utgifter för IT tjänster 2021'},
    inplace=True
    )
df2.head()
df1.head()  
df3.head()
# con = sql.connect('kunskapskontrol1.db')

# try:
#     df3.to_sql('data om it-utgifter och incidenter', con, if_exists='replace')
# except:
#     logger.debug(f'Loading to DB failed')
# else:
#     logger.info(f'Loading to DB successfull')

TypeError: Can only merge Series or DataFrame objects, a <class 'NoneType'> was passed

In [ ]:

# Kunskapskontrol 1
import pandas as pd
import sqlite3 as sql
import logging

logger = logging.getLogger(__name__)
logging.basicConfig(
        filename=r'C:\Users\vidar\Documents\Skóli\TUC\Data_Science\Projects\Kunskapskontrol_1\log.txt', 
        format='[%(asctime)s][%(levelname)s] %(message)s',
        level=logging.INFO
        )
# Läsa in csv från SCB, passa på att specifiera encoding som cp1252 annars får man inte svenska bokstaver

try:
    df1 = pd.read_csv(r'C:\Users\vidar\Documents\Skóli\TUC\Data_Science\Projects\Kunskapskontrol_1\000007N5_20241217-094027.csv', encoding='cp1252')
    df2 = pd.read_csv(r'C:\Users\vidar\Documents\Skóli\TUC\Data_Science\Projects\Kunskapskontrol_1\000003PS_20241217-093537.csv', encoding='cp1252')
except FileNotFoundError:
    logger.warning(f'File Not Found, check file path!')
else:
    logger.info(f'CSV file was read')
# rensa df1, droppa 2 kolumner som vi inte behöver samt alla nullvärde i den kolumnen vi håller i.
# SCB använder ".." som nullvärde så pandas .dropna kommer inte funka
df1 = df1.drop(
    ['typ av konsekvens','2023'], axis=1
    ).loc[df1['2021']!='..']
# i csv'en är siffrorna klassifierade som textsträngar så vi måste omvändla til int
df1['2021'] = df1['2021'].astype(int)
# Vi är egentligen bara intresserad av total incidenter
df1 = df1.groupby('redovisningsgrupp').sum()
# SCB använder ibland inte samma exakta namn för regionerna så vi får döpa om värden i första dataframe så de matchar namnen i den andra
def fix_regions(*dfs):
    for df in dfs:
        df.rename(
            index={'Stockholms län': 'Stockholm',
                   'Norra mellansverige':'Norra Mellansverige',
                   'Östra mellansverige':'Östra Mellansverige'},
            inplace=True
            )

fix_regions(df1, df2)

  
# För df2 kan vi droppa den kolumnen eftersom den bara har värdet "Totalt" i alla rader
df2.drop(['typ av IT-tjänst'], axis=1, inplace=True)
# Vi kör merge, som tar liknande argument som SQL join, och skapar df3
df3 = df1.merge(df2, 'left', right_on='redovisningsgrupp', left_on='redovisningsgrupp')

df3.rename(
    columns={'redovisningsgrupp':'Region',
             '2021_x':'Incidenter 2021',
             '2021_y':'Utgifter för IT tjänster 2021'},
    inplace=True
    )

con = sql.connect('kunskapskontrol1.db')

try:
    df3.to_sql('data om it-utgifter och incidenter', con, if_exists='replace')
except:
    logger.debug(f'Loading to DB failed')
else:
    logger.info(f'Loading to DB successfull')

In [14]:
import pandas as pd
import logging

logger = logging.getLogger(__name__)
logging.basicConfig(
        filename=r'C:\Users\vidar\Documents\Skóli\TUC\Data_Science\Projects\Kunskapskontrol_1\log.txt', 
        format='[%(asctime)s][%(levelname)s] %(message)s',
        level=logging.INFO
        )

def csv_reader():
    try:
        pd.read_csv(r'C:\Users\vidar\Documents\Skóli\TUC\Data_Science\Projects\Kunskapskontrol_1\000007N5_20241217-094027.csv', encoding='cp1252')
        pd.read_csv(r'C:\Users\vidar\Documents\Skóli\TUC\Data_Science\Projects\Kunskapskontrol_1\000003PS_20241217-093537.csv', encoding='cp1252')
    except FileNotFoundError:
        logger.warning(f'File Not Found, check file path!')
    else:
        logger.info(f'CSV file was read')
        


csv_reader()
        
        



In [6]:
import pandas as pd
import sqlite3 as sql
import logging

logger = logging.getLogger(__name__)
logging.basicConfig(
        filename=r'C:\Users\vidar\Documents\Skóli\TUC\Data_Science\Projects\Kunskapskontrol_1\log.txt', 
        format='[%(asctime)s][%(levelname)s] %(message)s',
        level=logging.INFO
        )
# Läsa in csv från SCB, passa på att specifiera encoding som cp1252 annars får man inte svenska bokstaver

try:
    df1 = pd.read_csv(r'C:\Users\vidar\Documents\Skóli\TUC\Data_Science\Projects\Kunskapskontrol_1\000007N5_20241217-094027.csv', encoding='cp1252')
    df2 = pd.read_csv(r'C:\Users\vidar\Documents\Skóli\TUC\Data_Science\Projects\Kunskapskontrol_1\000003PS_20241217-093537.csv', encoding='cp1252')
except FileNotFoundError:
    logger.warning(f'File Not Found, check file path!')
else:
    logger.info(f'CSV file was read')

      
def cleaner(df1, df2):
    df2.drop(['typ av IT-tjänst'], axis=1, inplace=True)
    df1 = df1.drop(['typ av konsekvens','2023'], axis=1).loc[df1['2021']!='..']
    df1['2021'] = df1['2021'].astype(int)
    df1 = df1.groupby('redovisningsgrupp').sum()
    return df1, df2
    
    
def fix_regions(df1):
        df1.rename(
            index={'Stockholms län': 'Stockholm',
                   'Norra mellansverige':'Norra Mellansverige',
                   'Östra mellansverige':'Östra Mellansverige'},
            inplace=True
                  )
        
def merger(df1, df2):  
    df3 = df1.merge(df2, 'left', right_on='redovisningsgrupp', left_on='redovisningsgrupp')
    df3.rename(columns={'redovisningsgrupp':'Region', '2021_x':'Incidenter 2021', '2021_y':'Utgifter för IT tjänster 2021'}, inplace=True)
    return df3

fix_regions
print(df1.head())
print(df2.head())




                                   typ av konsekvens    redovisningsgrupp  \
0  otillgängliga it-tjänster pga. fel i hårdvara/...       Stockholms län   
1  otillgängliga it-tjänster pga. fel i hårdvara/...  Östra mellansverige   
2  otillgängliga it-tjänster pga. fel i hårdvara/...    Småland med öarna   
3  otillgängliga it-tjänster pga. fel i hårdvara/...           Sydsverige   
4  otillgängliga it-tjänster pga. fel i hårdvara/...          Västsverige   

  2021 2023  
0   18   16  
1   15   23  
2   20   16  
3   18   15  
4   17   14  
  typ av IT-tjänst    redovisningsgrupp   2021
0           Totalt            Stockholm  36930
1           Totalt  Östra Mellansverige   6395
2           Totalt    Småland med öarna   7977
3           Totalt           Sydsverige   7105
4           Totalt          Västsverige  15656


In [ ]:
df3